In [432]:
# 돔트리
# CSS selector 한줄이라도 짧음.

# Crawling
- 텍스트 마이닝, 데이터 마이닝
- NLP 포커스 말뭉치 : 언어습관
- 실 데이터를 누적함.
- 세종21기반으로 형태소 분석기 만든게 카카오의 카이
- Systematically browses
- 구글 검색엔진의 시발점
- Focused Crawling을 할것 (원하는 범위까지만 정해진 방식으로 가져오기)

- 링크를 가져와서 재귀적으로 돌릴것임.
- URL List를 어떻게 관리 할것인지
- MultiThread, 분산처리가 필요함.
- 이미 Scrapy라는 패키지로 만들어져있기는 함. (이미 봇임).
- BeautifulSoup을 통해서 Dom객체 만들어서 parsing할 것임.
- 같은 내용인데 다른 url인지, 들어갔던건지, 걸러내기.
- 내부 저장부분은 html.text 저장하면 된다.

시드 주소 : http://example.webscraping.com/places/default/index

a tag에서 href가 항상 하이퍼링크 predefined 형식

In [1]:
import requests
import time

def download(method, url, 
             param = None, data = None, 
            timeout = 1,maxretries = 3):
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    #headers = None
    
    try:
        resp = requests.request(method, url,
                                params= param,
                                data=data, headers=headers)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0: 
            time.sleep(timeout) # param에따라 몇초 기다릴지 결정 할 수가 있다.
            print(maxretries) # 재귀적으로 자기 자신을 부르게 코드를 짜면 된다.
            resp = download(method,
                            url, param = param, data=data,
                            timeout=timeout,
                            maxretries = maxretries - 1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [2]:
from bs4 import BeautifulSoup
from requests import compat

# ------------------------------------------------------------

In [5]:
url = 'http://example.webscraping.com/places/default/index'
html = download("get",url)
dom = BeautifulSoup(html.text,"lxml")
dom.select("#mainFrame")

[]

In [14]:
def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text,"lxml")
    
    if len(dom.select("#mainFrame")) < 1:
        return []
    
    seed = compat.urljoin(seed,dom.select('#mainFrame')[0]['src'])
    html = download("get",seed)
    dom = BeautifulSoup(html.text,"lxml")
    
    return [compat.urljoin(html.url,_["href"])
                 for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(compat.urljoin(seed,_["href"]))]

In [15]:
queue = list()
queue.append('http://example.webscraping.com/places/default/index')

seen = list()

while True:
    baseURL = queue.pop(0) # FIFO
    seen.append(baseURL)
    
    time.sleep(0.2) # 너무 빠르면 안된다.
    
    linkList = parseURL(baseURL)
    for link in linkList:  # 중복되는 url은 빼주어야 한다. (Home 같은 것)
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue),len(seen)))
    # URL Seen : BaseURL에 쓰인 것을 다른 table에서 관리하다가 있으면 피한다.

Queue: 0, Seen: 1


IndexError: pop from empty list

In [25]:
def parseURL(seed):
    html = download("get",geturl(seed),getparams(seed))
    dom = BeautifulSoup(html.text,"lxml")

    if len(dom.select("#mainFrame")) < 1:
        return []
    
    seed = compat.urljoin(seed,dom.select('#mainFrame')[0]['src'])

    html = download("get",geturl(seed),getparams(seed))
    dom = BeautifulSoup(html.text,"lxml")

    return [compat.urljoin(html.url,_["href"])
                 for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(compat.urljoin(seed,_["href"]))]

In [26]:
def geturl(urlsrc):
    if '?' in urlsrc:
        return urlsrc.split('?')[0]+'?'
    else:
        return urlsrc

def getparams(urlsrc):
    if '?' in  urlsrc:
        try:
            paramList = urlsrc.split('?')[1].split('&')
            params = {_.split('=')[0]:compat.unquote(_.split('=')[1]) for _ in paramList}
            return params
        except:
            print(_)
    else:
        return None

In [27]:
queue = list()
queue.extend([_["href"] 
  for _ in dom2.select("a.sh_blog_title._sp_each_url._sp_each_title")
 if checkBlog(_["href"])])

seen = list()

# BFS Search (너비 우선 탐색)
while True:
    baseURL = queue.pop(0) # FIFO
    #baseURL = queue.pop(-1) # Stack으로 해서 깊이우선 탐색
    seen.append(baseURL)
    # URL Seen : BaseURL에 쓰인 것을 다른 table에서 관리하다가 있으면 피한다.
    
    time.sleep(0.5) # 너무 빠르면 안된다.
    
    linkList = parseURL(baseURL)
    for link in linkList:  # 중복되는 url은 빼주어야 한다. (Home 같은 것)
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue),len(seen)))

# Back of Add Model을 만들 수 있다.
# Or not vector(?) 
# 저차원으로 가면 Word Embedding


IndexError: pop from empty list

In [28]:
seen

[]

In [49]:
# NAVER
url = "https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query=%EB%B0%95%EB%B3%B4%EC%98%81"
param = getparams(url)
urlsrc = geturl(url)

html = download('get',urlsrc,param)
dom2 = BeautifulSoup(html.text,"lxml")

([_["href"] 
  for _ in dom2.select("a.sh_blog_title._sp_each_url._sp_each_title")
 if checkBlog(_["href"])]) # 네이버 블로그만 가져오기.

['https://blog.naver.com/cheawhi?Redirect=Log&logNo=221588720464',
 'https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950']

In [42]:
# DAUM
url = "https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q=%EB%B0%95%EB%B3%B4%EC%98%81"
param = getparams(url)
urlsrc = geturl(url)

html = download('get',urlsrc,param)
dom2 = BeautifulSoup(html.text,"lxml")

([_["href"] 
  for _ in dom2.select("#blogColl a.f_link_b")
 if checktistory(_["href"])]) # 다음 블로그만 가져오기.

['http://adam24eve.tistory.com/858',
 'http://bfh188.tistory.com/5',
 'http://clickplease.tistory.com/18',
 'http://listup.tistory.com/248']

In [39]:
def parseURL(seed):
    html = download("get",seed[0])
    dom = BeautifulSoup(html.text,"lxml")
    
    return [(compat.urljoin(seed[0],_["href"]),seed[1]+1) # 특정 depth까지만
                 for _ in dom.find_all("a") 
            if seed[1] < 2 and _.has_attr("href") and len(_["href"]) > 3
           and checktistory(compat.urljoin(seed[0],_["href"]))]

In [43]:
queue = list()
queue.extend([(_["href"] ,1) # url and depth
  for _ in dom2.select("#blogColl a.f_link_b")
 if checktistory(_["href"])])

seen = list()

while queue:
    
    baseURL = queue.pop(0) 
    seen.append(baseURL)
    
    time.sleep(0.1) # 너무 빠르면 안된다.
    
    linkList = parseURL(baseURL)
    for link in linkList:  # 중복되는 url은 빼주어야 한다. (Home 같은 것)
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue),len(seen)))

    
# Focused crawling은 얼마나 레퍼런스됬는지 볼때사용됨.

Queue: 45, Seen: 1
Queue: 68, Seen: 2
Queue: 81, Seen: 3
Queue: 105, Seen: 4
Queue: 104, Seen: 5
Queue: 103, Seen: 6
Queue: 102, Seen: 7
Queue: 101, Seen: 8
Queue: 100, Seen: 9
Queue: 99, Seen: 10
Queue: 98, Seen: 11
Queue: 97, Seen: 12
Queue: 96, Seen: 13
Queue: 95, Seen: 14
Queue: 94, Seen: 15
Queue: 93, Seen: 16
Queue: 92, Seen: 17
Queue: 91, Seen: 18
Queue: 90, Seen: 19
Queue: 89, Seen: 20
Queue: 88, Seen: 21
Queue: 87, Seen: 22
Queue: 86, Seen: 23
Queue: 85, Seen: 24
Queue: 84, Seen: 25
Queue: 83, Seen: 26
Queue: 82, Seen: 27
Queue: 81, Seen: 28
Queue: 80, Seen: 29
Queue: 79, Seen: 30
Queue: 78, Seen: 31
Queue: 77, Seen: 32
Queue: 76, Seen: 33
Queue: 75, Seen: 34
Queue: 74, Seen: 35
Queue: 73, Seen: 36
Queue: 72, Seen: 37
Queue: 71, Seen: 38
Queue: 70, Seen: 39
Queue: 69, Seen: 40
Queue: 68, Seen: 41
Queue: 67, Seen: 42
Queue: 66, Seen: 43
Queue: 65, Seen: 44
Queue: 64, Seen: 45
Queue: 63, Seen: 46
Queue: 62, Seen: 47
Queue: 61, Seen: 48
Queue: 60, Seen: 49
Queue: 59, Seen: 50
Que

In [41]:
def checktistory(url):
    return compat.urlparse(url)[1].find('tistory.com') != -1

In [45]:
queue

[]

In [46]:
for i in seen:
    print(i)

('http://adam24eve.tistory.com/858', 1)
('http://bfh188.tistory.com/5', 1)
('http://clickplease.tistory.com/18', 1)
('http://listup.tistory.com/248', 1)
('http://adam24eve.tistory.com/category', 2)
('http://adam24eve.tistory.com/category/Cosmos', 2)
('http://adam24eve.tistory.com/category/Mystery%20world', 2)
('http://adam24eve.tistory.com/category/Star', 2)
('http://adam24eve.tistory.com/category/Political%20', 2)
('http://adam24eve.tistory.com/category/It-game', 2)
('http://adam24eve.tistory.com/guestbook', 2)
('https://adam24eve.tistory.com/rss', 2)
('http://adam24eve.tistory.com/858', 2)
('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91', 2)
('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%EB%82%98%EC%9D%B4', 2)
('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%EB%B0%95%EB%B3%B4%EC%98%81%20%EC%97%B4%EC%95%A0', 2)
('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%ED%82%A4', 2)
('http://adam24eve.tistory.com/tag/%EB%AF%B8%

In [265]:
url = dom2.select("#blogColl a.f_link_b")[0]['href']

In [249]:
compat.urlparse(url)[1].find('tistory.com')

10

In [255]:
checkBlog2(url)

True

정규식으로 할거면 (.+?)[.]tistory.com

In [36]:
seed = ([_["href"] 
  for _ in dom2.select("a.sh_blog_title._sp_each_url._sp_each_title")
 if checkBlog(_["href"])])[0]
html = download("get",geturl(seed),param=getparams(seed))
dom = BeautifulSoup(html.text,"lxml")

IndexError: list index out of range

In [156]:
dom.select('#mainFrame')[0]['src']

'/PostView.nhn?blogId=imagetech1&logNo=221530358950&from=search&redirect=Log&widgetTypeCall=true&directAccess=false'

In [35]:
seed = compat.urljoin(geturl(seed),dom.select('#mainFrame')[0]['src'])

NameError: name 'seed' is not defined

In [50]:
dom2.select('#mainFrame')[0]['src']

IndexError: list index out of range

In [34]:
compat.urlparse('https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195')

ParseResult(scheme='https', netloc='blog.naver.com', path='/dlqlwm14', params='', query='Redirect=Log&logNo=221529807195', fragment='')

In [47]:
def checkBlog(url):
    return compat.urlparse(url)[1] == 'blog.naver.com'

In [52]:
checkBlog('https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195')

True

In [51]:
[_.find_parent()["href"] for _ in dom2.select(".LC20lb")][0]

IndexError: list index out of range

In [53]:
def parseURL(seed):
    html = download("get",geturl(seed),param=getparams(seed))
    dom = BeautifulSoup(html.text,"lxml")
    
    return [compat.urljoin(html.url,_["href"])
                 for _ in dom.find_all("a") 
            if _.has_attr("href") and not _["href"].startswith('#')]

In [102]:
def geturl(urlsrc):
    if '?' in urlsrc:
        return urlsrc.split('?')[0]+'?'
    else:
        return urlsrc

def getparams(urlsrc):
    if '?' in  urlsrc:
        paramList = urlsrc.split('?')[1].split('&')
        params = {_.split('=')[0]:compat.unquote(_.split('=')[1]) for _ in paramList}
        return params
    else:
        return None

In [54]:
queue = list()
#queue.append('https://www.google.com/search?newwindow=1&hl=en&source=hp&ei=wN8rXZW8NMn_8QWErqDgDg&q=박보영&oq=박보영&gs_l=psy-ab.3..0l10.2667.3629..3735...2.0..1.126.877.1j8......0....1..gws-wiz.....0..0i131.xCFY8trEo3I')
#for _ in ParkList:
#    queue.append(_[0])

queue.extend([_.find_parent()["href"] for _ in dom2.select(".LC20lb")])
seen = list()

# BFS Search (너비 우선 탐색)
while True:
    baseURL = queue.pop(0) # FIFO
    #baseURL = queue.pop(-1) # Stack으로 해서 깊이우선 탐색
    seen.append(baseURL)
    # URL Seen : BaseURL에 쓰인 것을 다른 table에서 관리하다가 있으면 피한다.
    
    time.sleep(1) # 너무 빠르면 안된다.
    
    linkList = parseURL(baseURL)
    for link in linkList:  # 중복되는 url은 빼주어야 한다. (Home 같은 것)
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue),len(seen)))

# Back of Add Model을 만들 수 있다.
# Or not vector(?) 
# 저차원으로 가면 Word Embedding


IndexError: pop from empty list

- 데이터베이스를 만드려면 테이블관의 관계를 어떻게 할건지 스키마 구성해야함
- 이미지만, 텍스트만 처리 혹은 구성할건지
- 데이터베이스는 PK가 제일 중요하다(Unique). 절대로 문자열 쓰지마라.  
DB는 빠르게 해쉬해야하는데 문자열이면 해쉬값을 못만든다.  
- ID , url, 언제 Access했는지(date값), seen인지 체크


- 테이블 1개에 id, url, seen, date가 필요
- url도 중복이 많이 생기기 때문에 참조하는 형태로 하면 속도가 빨라질 것이다.
- 

table1 = id, path, param seen, date  
table2 = id, netloc, date(timestamp 방식)  
url = netloc + path + param  

table1과 table2를 join시켜서 table2에 focused 크롤링 하면 된다.  

In [307]:
import sqlite3 as sql3

In [308]:
con = sql3.connect('myB.db',)

In [309]:
cur = con.cursor()

In [423]:
cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    table2_id INTEGER NOT NULL,
    path TEXT NOT NULL,
    param TEXT,
    depth INTEGER NOT NULL,
    inbound INTEGER NOT NULL,
    seen BOOLEAN DEFAULT FALSE,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    netloc TEXT NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""") # 나중에 본문 내용 빼놓고 이미지,텍스트로 분기도 가능하다.

In [424]:
con.commit()

In [425]:
def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text,"lxml")
    
    return [compat.urljoin(seed,_["href"]) # 특정 depth까지만
                 for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"]) > 3]

In [426]:
# Google
url = 'https://www.google.com/search?newwindow=1&ei=rhwsXfmJLsj58QXyyZugAQ&q=%EB%B0%95%EB%B3%B4%EC%98%81&oq=%EB%B0%95%EB%B3%B4%EC%98%81&gs_l=psy-ab.3..0l2j0i131j0l7.233632.234700..234750...1.0..1.137.847.1j7......0....1..gws-wiz.....0..0i71.NbxV7mwYbnE'
param = getparams(url)
urlsrc = geturl(url)

html = download('get',urlsrc,param)
dom = BeautifulSoup(html.text,"lxml")

In [427]:
for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
    
    _urlparse = compat.urlparse(href)
    netloc = "://".join(_urlparse[:2])
    cur.execute("SELECT id FROM table2 WHERE netloc = ? LIMIT 0,1"
                ,[netloc]) # netloc은 sequence로 만들어야함
    
    netlocID = cur.fetchone()
    if not netlocID:
        cur.execute("INSERT INTO table2 (netloc) VALUES(?)"
                    ,[netloc])
        con.commit()
        cur.execute("SELECT id FROM table2 WHERE netloc = ? LIMIT 0,1"
                ,[netloc])
        netlocID = cur.fetchone()
        
    cur.execute("INSERT INTO table1(table2_id,path,param,depth,inbound)\
    VALUES(?,?,?,?,?)",[netlocID[0],_urlparse[2],_urlparse[4],1,0])
    con.commit()
    print(cur.lastrowid,netlocID)
    

1 (1,)
2 (2,)
3 (2,)
4 (3,)
5 (4,)
6 (5,)
7 (6,)
8 (7,)
9 (8,)


In [428]:
i = 0
while True:
    cur.execute("""
    SELECT table1.id,table2.netloc,
           table1.path,table1.param,
           table1.depth, table2.id
    FROM table1
    JOIN table2 ON table1.table2_id=table2.id
    WHERE table1.seen = FALSE and table1.depth < 3
    ORDER BY table1.date ASC
    LIMIT 0,1;
    """)
    seed = cur.fetchone()
    if not seed:
        break;
    
    cur.execute("""
        UPDATE table1
        SET seen=TRUE
        where id = ?
    """,[seed[0]])
    con.commit()
    
    baseURL = "{0}{1}?{2}".format(seed[1],seed[2],seed[3]) # 주소 만들기
    for href in parseURL(baseURL):

        _urlparse = compat.urlparse(href)
        netloc = "://".join(_urlparse[:2])
        cur.execute("SELECT id FROM table2 WHERE netloc = ? LIMIT 0,1"
                    ,[netloc]) # netloc은 sequence로 만들어야함

        netlocID = cur.fetchone()
        if not netlocID:
            cur.execute("INSERT INTO table2 (netloc) VALUES(?)"
                        ,[netloc])
            con.commit()
            cur.execute("SELECT id FROM table2 WHERE netloc = ? LIMIT 0,1"
                    ,[netloc])
            netlocID = cur.fetchone()

        cur.execute("INSERT INTO table1(table2_id,path,param,depth,inbound)\
        VALUES(?,?,?,?,?)",[netlocID[0],_urlparse[2],_urlparse[4],seed[4]+1,
                           seed[5]])
        con.commit()
    i+=1
    if i==1000:
        break;

KeyboardInterrupt: 

In [404]:
cur.execute("INSERT INTO table1(table2_id,path,param)\
        VALUES(?,?,?)",[10,'',''])
con.commit()

In [359]:
[compat.urlparse(_.find_parent()["href"]) for _ in dom.select(".LC20lb")]
# params는 post방식에서 넘기는 것이다.

[ParseResult(scheme='https', netloc='namu.wiki', path='/w/%EB%B0%95%EB%B3%B4%EC%98%81', params='', query='', fragment=''),
 ParseResult(scheme='https', netloc='ko.wikipedia.org', path='/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', params='', query='', fragment=''),
 ParseResult(scheme='https', netloc='ko.wikipedia.org', path='/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%9E%91%ED%92%88_%EB%AA%A9%EB%A1%9D', params='', query='', fragment=''),
 ParseResult(scheme='https', netloc='news.joins.com', path='/article/22895953', params='', query='', fragment=''),
 ParseResult(scheme='https', netloc='movie.daum.net', path='/person/main', params='', query='personId=105244', fragment=''),
 ParseResult(scheme='http', netloc='m.cafe.daum.net', path='/parkboyoungfd', params='', query='', fragment=''),
 ParseResult(scheme='https', netloc='www.msn.com', path='/ko-kr/entertainment/tv/hd%EC%8A%A4%ED%86%A0%EB%A6%AC-%EC%9E%91%EC%9D%80-%EA%B1%B0%EC%9D%B8-%EB%B0%95%EB%B3%B4%EC%98%81%E2%80%A6%EA%B7%B8%EC%9D%98-%EC%97%B

In [346]:
cur.execute("SELECT id FROM table2 WHERE netloc = ? LIMIT 0,1"
                ,[netloc])
cur.fetchone()

(6,)

In [305]:
queue = list()
queue.extend([_.find_parent()["href"] for _ in dom.select(".LC20lb")])
seen = list()

while queue:
    baseURL = queue.pop(0) 
    seen.append(baseURL)
    
    time.sleep(0.1) # 너무 빠르면 안된다.
    
    linkList = parseURL(baseURL)
    for link in linkList:  # 중복되는 url은 빼주어야 한다. (Home 같은 것)
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue),len(seen)))

    
# Focused crawling은 얼마나 레퍼런스됬는지 볼때사용됨.

Queue: 45, Seen: 1
Queue: 61, Seen: 2
Queue: 85, Seen: 3
Queue: 92, Seen: 4
Queue: 91, Seen: 5
Queue: 90, Seen: 6
Queue: 89, Seen: 7
Queue: 88, Seen: 8
Queue: 87, Seen: 9
Queue: 86, Seen: 10
Queue: 85, Seen: 11
Queue: 84, Seen: 12
Queue: 83, Seen: 13
Queue: 82, Seen: 14
Queue: 81, Seen: 15
Queue: 80, Seen: 16
Queue: 79, Seen: 17
Queue: 78, Seen: 18
Queue: 77, Seen: 19
Queue: 76, Seen: 20
Queue: 75, Seen: 21
Queue: 74, Seen: 22
Queue: 73, Seen: 23
Queue: 72, Seen: 24
Queue: 71, Seen: 25
Queue: 70, Seen: 26
Queue: 69, Seen: 27
Queue: 68, Seen: 28
Queue: 67, Seen: 29
Queue: 66, Seen: 30
Queue: 65, Seen: 31
Queue: 64, Seen: 32
Queue: 63, Seen: 33
Queue: 62, Seen: 34
Queue: 61, Seen: 35
Queue: 60, Seen: 36
Queue: 59, Seen: 37
Queue: 58, Seen: 38
Queue: 57, Seen: 39
Queue: 56, Seen: 40
Queue: 55, Seen: 41
Queue: 54, Seen: 42
Queue: 53, Seen: 43
Queue: 52, Seen: 44
Queue: 51, Seen: 45
Queue: 50, Seen: 46
Queue: 49, Seen: 47
Queue: 48, Seen: 48
Queue: 47, Seen: 49
Queue: 46, Seen: 50
Queue: 45

In [332]:
compat.urlparse(a[0])

ParseResult(scheme='https', netloc='namu.wiki', path='/w/%EB%B0%95%EB%B3%B4%EC%98%81', params='', query='', fragment='')

In [ ]:
select inbound,count(inbound)
from table1 where table2_id=2 group by inbound;

In [430]:
cur.execute("""
    select inbound,count(inbound)
    from table1 
    where table2_id=10 group by inbound;
""")
cur.fetchall() # outbound 보기 가능.

[(1, 979), (10, 1310)]

In [431]:
cur.execute("""
    select count(inbound)
    from table1 
    where inbound=1;
""")
cur.fetchall()

[(35768,)]

- 979/35768이 1번 사이트의 가중치이다.
- 정보의 중요성이 얼마나 되는지 볼 때 inbound, outbound를 이용한 rank를 사용

- 내일은 contents를 담는 table만들어서 엮을 것임.
- 원하는 data 뽑아서 넣을 것임